# Imports

In [1]:
import numpy as np
import pandas as pd
import sklearn
import joblib
from sklearn.preprocessing import StandardScaler, MinMaxScaler, normalize
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
import matplotlib.pyplot as plt
from matplotlib.ticker import MaxNLocator
from tqdm import tqdm
import os
import sys
import tensorflow as tf
import pathlib
from pathlib import Path

from utils_gianluca_copy import *
from utils_torch import *

In [2]:
def r2(y_pred,y_test):
    return r2_score(y_test, y_pred)

def mae(y_pred,y_test):
    return mean_absolute_error(y_test, y_pred)

def mse(y_pred,y_test):
    return mean_squared_error(y_test, y_pred)

def rmse(y_pred,y_test):
    return np.sqrt(mean_squared_error(y_test, y_pred))

# Simulated data

## Load dataset

In [3]:
# read the simulated dataset.
#path = Path(os.getcwd()).parent.__str__()
ds = pd.read_parquet('Dataset_Gianluca/ds_sim.parquet')

In [4]:
CASE = "_3000" # _10, _100, _1000, _3000
case = int(CASE.split("_")[-1])
length = case/10
print(case, length)

3000 300.0


In [10]:
OVERWRITE = False

# Chop each multivariate time series, associated to each couple (drive cycle, soh), to shorter time windows
# X_idxs: a Nx2 matrix, whose i-th row is the start and end index of the i-th time window of the new tw dataset in the original dataset
# y: soh for each extracted time window
#X_idxs, y = extract_time_windows(ds, length=int(length), freq=10, random_starting_point=False, verbose=True, random_state=42, overwrite=OVERWRITE, suffix=CASE)
X_idxs, y = extract_time_windows(ds, length=int(length), freq=10, random_starting_point=False, verbose=True, random_state=42, overwrite=OVERWRITE, suffix=CASE)
print(f'Extracted time windows: {len(y)}')

100%|██████████| 189/189 [00:26<00:00,  7.10it/s]

Extracted time windows: 9013


In [ ]:
ds.iloc[X_idxs[10][0]:X_idxs[10][1]]

In [ ]:
idx_train, idx_test, y_train, y_test = train_test_split(np.arange(len(y)), y, test_size=0.2, stratify=y, random_state=42)
X_idxs_train, X_idxs_test = X_idxs[idx_train], X_idxs[idx_test]

In [ ]:
# Display example of extracted windows
ds.iloc[X_idxs_train[10][0]:X_idxs_train[10][1]][["Timestamp", "Voltage","Current", "SOC", "Temperature"]]
#ds.iloc[X_idxs_train[10][0]:X_idxs_train[10][1]][["Voltage","Current", "SOC", "Temperature"]].values
#ds.iloc[X_idxs_train[10][0]:X_idxs_train[10][1]][["Voltage","Current", "SOC", "Temperature"]].values.shape
#y_train[10]

In [ ]:
# Fit a StandardScaler object on the training set and then standardize the whole dataset with the found per-predictor mean and std; keep only V, I, SOC
X_idxs_train_concatenated = np.concatenate([np.arange(start,end) for start,end in X_idxs_train])
scaler = StandardScaler()
scaler.fit(ds[['Voltage','Current','SOC', 'Temperature']].iloc[X_idxs_train_concatenated])
ds_preproc = pd.DataFrame(scaler.transform(ds[['Voltage','Current','SOC', 'Temperature']]), \
                          index=ds.index, columns=['Voltage','Current','SOC', 'Temperature'])

In [ ]:
ds_preproc

In [ ]:
# Popolate training set.
X_train = np.zeros((len(X_idxs_train), case, 4))
Y_train = np.zeros((len(X_idxs_train), 1))

for i, idx in enumerate(X_idxs_train):
    X_train[i,:,:] = ds_preproc.iloc[idx[0]:idx[1]][["Voltage","Current", "SOC", "Temperature"]].values
    Y_train[i] = y_train[i]
    
#np.save(f"X_train{CASE}.npy", X_train)
#np.save(f"Y_train{CASE}.npy", Y_train)

In [ ]:
# Popolate test set.
X_test = np.zeros((len(X_idxs_test), case, 4))
Y_test = np.zeros((len(X_idxs_test), 1))

for i, idx in enumerate(X_idxs_test):
    X_test[i,:,:] = ds_preproc.iloc[idx[0]:idx[1]][["Voltage","Current", "SOC", "Temperature"]].values
    Y_test[i] = y_test[i]
    
#np.save(f"X_test{CASE}.npy", X_test)
#np.save(f"Y_test{CASE}.npy", Y_test)

In [ ]:
# ... or load npy files.
X_train = np.load(f"X_train{CASE}.npy")
Y_train = np.load(f"Y_train{CASE}.npy")
X_test = np.load(f"X_test{CASE}.npy")
Y_test = np.load(f"Y_test{CASE}.npy")

## Training basic models.

### NN

In [ ]:
batch_size = 256
epochs = 400
patience = 50

inputs = tf.keras.layers.Input((int(case),4))
x = tf.keras.layers.Flatten()(inputs)
#x = tf.keras.layers.Conv1D(32, 3, activation='relu', input_shape=[int(case),4])(x)
#x = tf.keras.layers.Dense(128, activation="relu")(x)
x = tf.keras.layers.Dense(64, activation="relu")(x)
x = tf.keras.layers.Dense(32, activation="relu")(x)
x = tf.keras.layers.Dropout(0.2)(x)
outputs = tf.keras.layers.Dense(1, activation='sigmoid')(x)

model = tf.keras.Model(inputs=[inputs], outputs=outputs)
model.compile(optimizer='adam', loss='mean_squared_error')

tmp_folder_path = os.sep.join([os.getcwd(), "weights", CASE.split("_")[-1]])
Path(pathlib.PureWindowsPath(tmp_folder_path).as_posix()).mkdir(parents=True, exist_ok=True)

early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss', 
                                                  patience=patience, 
                                                  restore_best_weights=True)

checkpoint = tf.keras.callbacks.ModelCheckpoint(filepath=os.sep.join([os.getcwd(), "weights", \
                                                                      CASE.split("_")[-1], "NN_weights.h5"]), 
                                                save_best_only=True,
                                                monitor='val_loss',
                                                mode='min',
                                                save_weights_only=True)

model.fit(X_train, Y_train,
          validation_data=(X_test, Y_test),
          batch_size=batch_size,
          epochs=epochs,
          callbacks=[early_stopping, checkpoint])


In [ ]:
preds = model.predict(X_test)
accuracy = np.equal(Y_test, np.around(preds,2)).sum() / Y_test.shape[0]
preds[:10]

In [ ]:
dict = model.evaluate(X_test, Y_test)
#model.history.history['loss']
dict

In [ ]:
out=open('risultati.txt', 'a')

out.write(f'\nBatch size:{batch_size}\t\tEpochs: {epochs}\t\tPatience: {patience}\n')
# lista_di_metriche = [f'{key}:{value}\n' for key, value in model.get_metrics_result().items()]
# for l in lista_di_metriche:
#     out.write(l)
out.write(f'Accuracy: {accuracy}\n')
out.write(f"RMSE: {rmse(preds,Y_test)}\n")
out.write(f"MAE {mae( preds,Y_test)}\n")
out.write(f"MSE: {mse( preds,Y_test)}\n")
out.write(f"R2: {r2(preds,Y_test)}\n")

#results = pd.DataFrame([y_test,preds]).T.rename(columns={0:'Real',1:'Predicted'})
#out.write(results.to_string())
out.write('\n--------------------------------------------------------------\n')

In [ ]:
tf.keras.backend.clear_session()

### Random forest

In [ ]:
nsamples, nx, ny = X_train.shape
d2_train_dataset = X_train.reshape((nsamples,nx*ny))
y_train = Y_train.ravel()
nsamples, nx, ny = X_test.shape
d2_test_dataset = X_test.reshape((nsamples,nx*ny))
y_test = Y_test.ravel()

In [ ]:
rfr = RandomForestRegressor(n_estimators=100,
                            max_depth=100,     # None, 5, 15, 30, 50, 100
                            max_features=3,     # 2, 3
                            bootstrap=True,     # True, False
                            max_samples=0.75,    # 0.5, 0.625, 0.75, 0.875, 1 (None when bootstrap=False)
                            random_state=42,
                            n_jobs=-1,
                            warm_start=False)

rfr.fit(d2_train_dataset, y_train)

In [ ]:
# Save RF model.
joblib.dump(rfr, os.sep.join([os.getcwd(), "weights", CASE.split("_")[-1], "RF_weights.joblib"]))

In [ ]:
y_pred = rfr.predict(d2_test_dataset)

In [ ]:
from scipy import stats

In [ ]:
results = pd.read_csv(os.sep.join([os.getcwd(), "Risultati ML", "10", "Simulated", "ML", "RF", "predictions.csv"]), index_col=[0])

In [ ]:
print("RMSE",rmse(results["Real"],results["Predicted"])*100)
print("MAE", mae(results["Real"],results["Predicted"])*100)
#print("MSE", mse(results["Real"],results["Predicted"]))
#print(f"R2: {stats.linregress(results["Real"], results["Predicted"]).rvalue**2:.6f}")
print("R2", r2(results["Real"],results["Predicted"]))

In [ ]:
results["Real"].values

In [ ]:
print("RMSE",rmse(y_pred,y_test))
print("MAE", mae(y_pred,y_test))
print("MSE", mse(y_pred,y_test))
print("R2", r2(y_pred,y_test))

results = pd.DataFrame([y_test,y_pred]).T.rename(columns={0:'Real',1:'Predicted'})
results.to_csv(os.sep.join([os.getcwd(), "Risultati ML", CASE.split("_")[-1], "Simulated", "ML", "RF", "predictions.csv"]))

## Real data

In [ ]:
# read the dataset
#path = Path(os.getcwd()).parent.__str__()
#ds_aviloo = pd.read_parquet(os.sep.join([path, 'ds_aviloo.parquet']))
ds_aviloo = pd.read_parquet('Dataset_Gianluca/ds_aviloo.parquet')
ds_aviloo_preproc = pd.DataFrame(scaler.transform(ds_aviloo[['Voltage','Current','SOC','Temperature']]), \
                                 index=ds_aviloo.index, columns=['Voltage','Current','SOC', 'Temperature'])

In [ ]:
ds_aviloo_preproc.shape

In [ ]:
ds_aviloo_preproc

In [ ]:
OVERWRITE = True

# Chop each multivariate time series, associated to each couple (drive cycle, soh), to shorter time windows
# X_idxs: a Nx2 matrix, whose i-th row is the start and end index of the i-th time window of the new tw dataset in the original dataset
# y: soh for each extracted time window
X_idxs_aviloo, y_aviloo = extract_time_windows(ds_aviloo, length=int(length), freq=10, random_starting_point=False, verbose=True, random_state=42, overwrite=OVERWRITE, suffix='_aviloo'+CASE)
print(f'Extracted time windows: {len(y_aviloo)}')

In [ ]:
# Remove time windows with NaN values (especially in Temperature).
remove = []
for i, idx in enumerate(X_idxs_aviloo):
    if np.any(np.isnan(ds_aviloo_preproc.iloc[idx[0]:idx[1]][["Voltage","Current", "SOC", "Temperature"]].values)):
        remove.append(i)
        
X_idxs_aviloo = np.delete(X_idxs_aviloo, remove, 0)
y_aviloo = np.delete(y_aviloo, remove, 0)

In [ ]:
# Popolate real test set.

X_test_aviloo = np.zeros((len(X_idxs_aviloo), int(case), 4))
Y_test_aviloo = np.zeros((len(X_idxs_aviloo), 1))

for i, idx in enumerate(X_idxs_aviloo):
    X_test_aviloo[i,:,:] = ds_aviloo_preproc.iloc[idx[0]:idx[1]][["Voltage","Current", "SOC", "Temperature"]].values
    Y_test_aviloo[i] = y_aviloo[i]

np.save(f"X_test_aviloo{CASE}.npy", X_test_aviloo)
np.save(f"y_test_aviloo{CASE}.npy", Y_test_aviloo)

In [ ]:
ds_aviloo_preproc.iloc[X_idxs_aviloo[118346][0]:X_idxs_aviloo[118346][1]][["Voltage","Current", "SOC", "Temperature"]]

In [ ]:
# ... or load Numpy arrays.
X_test_aviloo = np.load(f"X_test_aviloo_{CASE}.npy")
Y_test_aviloo = np.load(f"y_test_aviloo_{CASE}.npy")

### Test real data on NN

In [ ]:
model.load_weights(os.sep.join([os.getcwd(), "weights", CASE.split("_")[-1], "NN_weights.h5"]))
preds_aviloo = model.predict(X_test_aviloo)

In [ ]:
print("RMSE",rmse(preds_aviloo,Y_test_aviloo))
print("MAE", mae(preds_aviloo,Y_test_aviloo))
print("MSE", mse(preds_aviloo,Y_test_aviloo))
print("R2", r2(preds_aviloo,Y_test_aviloo))

results = pd.DataFrame([Y_test_aviloo.ravel(), preds_aviloo.ravel()]).T.rename(columns={0:'Real',1:'Predicted'})
results.to_csv(os.sep.join([os.getcwd(), "Risultati ML", CASE.split("_")[-1], "Real", "ML", "NN", "predictions.csv"]))

### Test real data on RF

In [ ]:
nsamples, nx, ny = X_test_aviloo.shape
d2_test_dataset_aviloo = X_test_aviloo.reshape((nsamples,nx*ny))
y_test_aviloo = Y_test_aviloo.ravel()

In [ ]:
y_pred = rfr.predict(d2_test_dataset_aviloo)

In [ ]:
print("RMSE",rmse(y_pred,y_test_aviloo))
print("MAE", mae(y_pred,y_test_aviloo))
print("MSE", mse(y_pred,y_test_aviloo))
print("R2", r2(y_pred,y_test_aviloo))

results = pd.DataFrame([y_test_aviloo,y_pred]).T.rename(columns={0:'Real',1:'Predicted'})
results.to_csv(os.sep.join([os.getcwd(), "Risultati ML", CASE.split("_")[-1], "Real", "ML", "RF", "predictions.csv"]))

# Transfer learning

In [ ]:
idx_train_aviloo, idx_test_aviloo, y_train_aviloo, y_test_aviloo = train_test_split(np.arange(len(y_aviloo)), y_aviloo, \
                                                                                    test_size=0.7, stratify=y_aviloo, \
                                                                                    random_state=42)
X_idxs_train_aviloo, X_idxs_test_aviloo = X_idxs_aviloo[idx_train_aviloo], X_idxs_aviloo[idx_test_aviloo]

In [ ]:
# Popolate training set.
X_train_aviloo_TF = np.zeros((len(X_idxs_train_aviloo), int(case), 4))
Y_train_aviloo_TF = np.zeros((len(X_idxs_train_aviloo), 1))

for i, idx in enumerate(X_idxs_train_aviloo):
    X_train_aviloo_TF[i,:,:] = ds_aviloo_preproc.iloc[idx[0]:idx[1]][["Voltage","Current", "SOC", "Temperature"]].values
    Y_train_aviloo_TF[i] = y_train_aviloo[i]
    
np.save(f"X_train_aviloo_TF{CASE}.npy", X_train_aviloo_TF)
np.save(f"Y_train_aviloo_TF{CASE}.npy", Y_train_aviloo_TF)

In [ ]:
# Popolate test set.
X_test_aviloo_TF = np.zeros((len(X_idxs_test_aviloo), int(case), 4))
Y_test_aviloo_TF = np.zeros((len(X_idxs_test_aviloo), 1))

for i, idx in enumerate(X_idxs_test_aviloo):
    X_test_aviloo_TF[i,:,:] = ds_aviloo_preproc.iloc[idx[0]:idx[1]][["Voltage","Current", "SOC", "Temperature"]].values
    Y_test_aviloo_TF[i] = y_test_aviloo[i]
    
np.save(f"X_test{CASE}.npy", X_test_aviloo_TF)
np.save(f"Y_test{CASE}.npy", Y_test_aviloo_TF)

In [ ]:
Y_test_aviloo_TF.shape

## TF NN

In [ ]:
batch_size = 256
epochs = 1000
patience = 100

early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss', 
                                                  patience=patience, 
                                                  restore_best_weights=True)

checkpoint = tf.keras.callbacks.ModelCheckpoint(filepath=os.sep.join([os.getcwd(), "weights", \
                                                                      CASE.split("_")[-1], "TF_NN_weights.h5"]), 
                                                save_best_only=True,
                                                monitor='val_loss',
                                                mode='min',
                                                save_weights_only=True)

model.load_weights(os.sep.join([os.getcwd(), "weights", CASE.split("_")[-1], "NN_weights.h5"]))

model.fit(X_train_aviloo_TF, Y_train_aviloo_TF,
          validation_data=(X_test_aviloo_TF, Y_test_aviloo_TF),
          batch_size=batch_size,
          epochs=epochs,
          callbacks=[early_stopping, checkpoint])

In [ ]:
preds_TF_NN = model.predict(X_test_aviloo_TF)

In [ ]:
print("RMSE",rmse(preds_TF_NN,Y_test_aviloo_TF))
print("MAE", mae(preds_TF_NN,Y_test_aviloo_TF))
print("MSE", mse(preds_TF_NN,Y_test_aviloo_TF))
print("R2", r2(preds_TF_NN,Y_test_aviloo_TF))

results = pd.DataFrame([Y_test_aviloo_TF.ravel(), preds_TF_NN.ravel()]).T.rename(columns={0:'Real',1:'Predicted'})
results.to_csv(os.sep.join([os.getcwd(), "Risultati ML", CASE.split("_")[-1], "Real", "TL", "NN", "predictions.csv"]))

## RF

In [ ]:
rfr = joblib.load(os.sep.join([os.getcwd(), "weights", CASE.split("_")[-1], "RF_weights.joblib"]))
rfr.warm_start=True
rfr.n_estimators+= 32
print(rfr.n_estimators)

In [ ]:
# Reshape RF dataset from 3D to 2D.
nsamples, nx, ny = X_train_aviloo_TF.shape
d2_train_dataset_tl = X_train_aviloo_TF.reshape((nsamples,nx*ny))
y_train_tl = Y_train_aviloo_TF.ravel()
nsamples, nx, ny = X_test_aviloo_TF.shape
d2_test_dataset_tl = X_test_aviloo_TF.reshape((nsamples,nx*ny))
y_test_tl = Y_test_aviloo_TF.ravel()

In [ ]:
rfr.fit(d2_train_dataset_tl, y_train_tl)

In [ ]:
preds_TF_RF = rfr.predict(d2_test_dataset_tl)

In [ ]:
print("RMSE",rmse(preds_TF_RF, y_test_tl))
print("MAE", mae(preds_TF_RF, y_test_tl))
print("MSE", mse(preds_TF_RF, y_test_tl))
print("R2", r2(preds_TF_RF, y_test_tl))

results = pd.DataFrame([y_test_tl, preds_TF_RF]).T.rename(columns={0:'Real',1:'Predicted'})
results.to_csv(os.sep.join([os.getcwd(), "Risultati ML", CASE.split("_")[-1], "Real", "TL", "RF", "predictions.csv"]))